In [1]:
""" Reading in md file """

with open("Data/interview-prep-questions-with-answers.md", "r") as input_file:
    text = input_file.read()

text= text.replace('###', ' ^')
t= text.split('\n##')

In [2]:
# import re
# pattern= r'\*(\w+)\*'

"""Modules list, questions, and answer parsing""" 

interview_dict= {} #dictionary for questions and answers only 
module_list= []
module_dict_index= []
count=0 #To obtain module index

for x in t:
    
    """ Replacing values For Proper Parsing"""

    mod= x.replace('**', '||') #there are words in between **word** that I don't want to split
    # mod = re.sub(pattern, r'\1', x)
    mod= mod.replace('*national*', 'national') #this word kept splitting 
    mod= mod.replace('*variable*', 'variable') 
    mod= mod.replace("You're looking at your Pandas Dataframe and all data appears to be either an int or a float.   \n    * How can you confirm your datatypes?", "You're looking at your Pandas Dataframe and all data appears to be either an int or a float. How can you confirm your datatypes?")
    mod= mod.replace('\n * Why do design', '\n* Why do design') 
    mod= mod.replace('Select * ', 'Select *')
    mod= mod.replace("*High reliability*", "'High reliability'" )
    mod= mod.replace('a* ', 'a*')
    mod= mod.replace('a.* ', 'a.*')
    mod= mod.replace("*Load*", "'Load'")
    mod= mod.replace("*Extract*", "'Extract'")
    mod= mod.replace("*Transform*", "'Transform'")
    mod= mod.replace("ordered*", "'ordered'")
    mod= mod.replace("*Load balancing*", 'Load balancing')
    mod= mod.replace("*except*", "'except'")
    mod= mod.replace('[Return to Home](../README.md)', '')

    
    mod= mod.replace(' * ', '@') #replacement for answers
    q_a= mod.replace('*', '$*') #replacement for questions
    
    """ Creating a list of modules"""

    if mod[1:7] == "Module" or mod[1:7] == "Genera" :
        mod= mod.split('*')
        module= mod[0].replace('\n','')

        #Removing subcategories from modules
        m_index= [i for i,e in enumerate(module) if e == '^']
        try:
            module_list.append(module[0:m_index[0]])
        except IndexError as e:
            module_list.append(module)
        
    """Parsing through questions and answers"""
    
    if isinstance(mod, list):
        q_a= q_a.split('$* ')[1:]
        q_a= [ q_a[x].replace('$*', '*') for x in range(len(q_a)) ] # To avoid wrong splitting
        qa= [x.split('* ') for x in q_a] # questions and answers per module

        #Breaking up question 3 from module 4. It is formatted differently
        if x == t[4]:
            
            #indexes
            module4_q_list= [1,4,9,11]
            module4_ans_list= [4,9,11,15]
            
            for x in range(len(module4_q_list)):
                q= f"{qa[2][0].split('@')[0]} {qa[2][0].split('@')[module4_q_list[x]]}"
                ans= f"{qa[2][0].split('@')[(module4_q_list[x] + 1): module4_ans_list[x]]}"
                # print(f'{q}: {ans}')

                #Build Dictionary
                interview_dict[f'Question: {q}'] = ans
                
                #Module Index
                module_dict_index.append(count)

        else:
            for x in range(len(qa)):
                ans= qa[x][0].split('@')[1:]
                q= qa[x][0].split('@')[0]
                # print(f'{q}: {ans}')
                
                #Build Dictionary
                interview_dict[f'Question: {q}'] = ans

                #Module index
                module_dict_index.append(count)
        
        count = count + 1


In [3]:
"""Building dictionary categorized by modules"""

categorized_dict = {}
modules=[]
for x in module_dict_index: 
    modules.append(module_list[x])
    

for module, (question, answers) in zip(modules, interview_dict.items()):
    if module not in categorized_dict:
        categorized_dict[module] = {}
    categorized_dict[module][question] = answers

# categorized_dict[' Module 1 - Relational Databases ']

In [4]:
"""Creating Dataframe from the categorized dictionary"""

import pandas as pd 

interview_data = []
general_data= []
for module, questions in categorized_dict.items():
    for question, answers in questions.items():
        try:
            interview_data.append([module, question, answers[0]])
        except IndexError as e:
            interview_data.append([module, question, answers[0]])
            

df= pd.DataFrame(data= interview_data, columns=['Module', 'Questions', 'Answers'])
df

,Module,Questions,Answers
0,Module 1 - Relational Databases,"Question: What makes relational databases ""rel...",A relational database stores associated inform...
1,Module 1 - Relational Databases,Question: What types of relationship can exist...,Tables can have one of the following 3 (4) rel...
2,Module 1 - Relational Databases,Question: What is an ERD?\n,An entity-relationship diagram (ERD) is a visu...
3,Module 1 - Relational Databases,Question: What is normalization?\n,Normalization is the process of applying best-...
4,Module 1 - Relational Databases,Question: What is denormalization?\n,Denormalization is the deliberate relaxing of ...
...,...,...,...
129,Modules 10 & 11 - Kafka & Cloud ETL and Appli...,Question: How do you run a pipeline in Azure D...,"In ADF, open the pipeline, click either ""Debug..."
130,Modules 10 & 11 - Kafka & Cloud ETL and Appli...,"Question: ""Systems Design"" - What are some res...",Most of my familiarity is with the Azure cloud...
131,General,Question: What are the stages you'd take in a ...,"Gather data, clean the data, load the data, ex..."
132,General,Question: How would you choose which tool to u...,Some solutions may be easier to use depending ...


In [5]:
df.loc[df['Module'].str.contains('Module 1')]

,Module,Questions,Answers
0,Module 1 - Relational Databases,"Question: What makes relational databases ""rel...",A relational database stores associated inform...
1,Module 1 - Relational Databases,Question: What types of relationship can exist...,Tables can have one of the following 3 (4) rel...
2,Module 1 - Relational Databases,Question: What is an ERD?\n,An entity-relationship diagram (ERD) is a visu...
3,Module 1 - Relational Databases,Question: What is normalization?\n,Normalization is the process of applying best-...
4,Module 1 - Relational Databases,Question: What is denormalization?\n,Denormalization is the deliberate relaxing of ...
5,Module 1 - Relational Databases,Question: What is a primary key?\n,A primary key is a column or group of columns ...
6,Module 1 - Relational Databases,Question: What is a foreign key?\n,A foreign key is a table's reference to anothe...
7,Module 1 - Relational Databases,Question: What is SQL?\n,Structured Query Language (SQL) is programming...
8,Module 1 - Relational Databases,Question: Is SQL case-sensitive?\n,"No, SQL is not case-sensitive; however, the co..."
9,Module 1 - Relational Databases,Question: Is SQL whitespace-sensitive?\n,"No, SQL is not whitespace-sensitive. Line brea..."


In [6]:
df.to_csv('Data/interview_qa.csv', sep= '~')